# Define the Area

## In this first notebook, we demonstrate how to define the location of interest and relevant time period to extract relevant altimeter data

## Dependencies

In [1]:
import numpy as np
import pandas as pd
import netCDF4
from netCDF4 import Dataset as NetCDFFile 
from netCDF4 import num2date, date2num, date2index
import datetime as dt
# from StringIO import StringIO
# import xarray as xr
import re

## A. Extract altimeter data

### Download list of altimeter URLS from Australian Ocean Data Network (AODN) as a txt file, here named IMOSURLS.txt

In [2]:
def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [3]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [4]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

## B. Define area and time

In [5]:
#Example latitude and longitude
latmin = -27.0
latmax = -10.0  
lonmin = 143.0
lonmax = 155.0 

if latmin>latmax:
    print('Error wrong definition of min and max lat')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon')

#Example time
start_date = dt.datetime(1985,1,1) 
end_date = dt.datetime(2018,12,31)

## Arguments

- lat: latitude
- lon: longitude
- wh: significant wave height
- t: time
- qc: quality control flag, one represents "very good data" (Ribal and Young 2019)
- back: backscatter
- ws: windspeed

## C. Define arguments based on time and spatial location

In [9]:
boxLat = [] 
boxLon = [] 
boxWh = [] 
boxT = [] 
boxQ = [] 
boxB = [] 
boxWS = [] 


newday_it = 0

for u in range(len(allURL)):
    urlON = allURL[u]
    for k in range(len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        ws = ncs.variables['WSPD_CAL'][:]
        
        
        if u == 2: #Saral altimeter
            wh = ncs.variables['SWH_KA_CAL'][:]
            qc = ncs.variables['SWH_KA_quality_control'][:]
            back = ncs.variables['SIG0_KA'][:]
        
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]
            qc = ncs.variables['SWH_KU_quality_control'][:]
            back = ncs.variables['SIG0_KU'][:]
             
        # Get desired time interval  
        time_var = ncs.variables['TIME']
        tt = ncs.variables['TIME'][:]
        timing = netCDF4.num2date(tt,time_var.units)

        newday_it = 0
        if lats.min() >= latmin and lats.min() <= latmax:
            if lons.min() >= lonmin and lons.min() <= lonmax:
                for p in range(len(timing)):
                    if timing[p] >= start_date and timing[p] <= end_date:
                        if wh[p]>0:
                            if qc[p]==1:
                                t1 = netCDF4.num2date(tt[p],u'days since 1985-01-01 00:00:00 UTC')

                                if newday_it == 0:
                                    dd = netCDF4.num2date(tt[p],u'days since 1985-01-01 00:00:00 UTC')
                                    newday_it = p
                                else:
                                    if t1.day != dd.day:

                                        dd = netCDF4.num2date(tt[p],u'days since 1985-01-01 00:00:00 UTC')
                                        tmpwh = wh[newday_it:p]
                                        tmpback = back[newday_it:p]
                                        tmpws = ws[newday_it:p]
                                        tmpqc = qc[newday_it:p]
                                        tmplat = lats[newday_it:p]
                                        tmplon = lons[newday_it:p]
                                        tmptt = tt[newday_it:p]
                                        
                                        iddd = np.where(np.logical_and(tmpqc==1,tmpwh>=0))
                                        boxWh.append(np.median(tmpwh[iddd]))
                                        boxB.append(np.median(tmpback[iddd]))
                                        boxWS.append(np.median(tmpws[iddd]))
                                        boxQ.append(np.median(tmpqc[iddd]))
                                        boxLat.append(np.median(tmplat[iddd]))
                                        boxLon.append(np.median(tmplon[iddd]))
                                        boxT.append((tmptt[0])) 
                                        newday_it = p


In [ ]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
        qc = boxQ[k]
        back = boxB[k]
        ws = boxWS[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])
        qc = np.append(qc,boxQ[k])
        back = np.append(back,boxB[k])
        ws = np.append(ws,boxWS[k])

## D. Create dataframe and file

In [ ]:
df = pd.DataFrame(
    {'lat':lat.flatten(),
     'lon':lon.flatten(),
     'wh':wh.flatten(),
     'tt':tt.flatten(),
     'qc':qc.flatten(),
     'back':back.flatten(),
     'ws':ws.flatten()
    })
nameCSV = 'filename.csv'
df.to_csv(str(nameCSV),columns=['lat', 'lon', 'wh', 'tt', 'qc', 'back','ws'], sep=' ', index=False ,header=1)